In [1]:
import sys
sys.path.insert(0, '../src')

from cesiumjs_anywidget import CesiumWidget
import ipywidgets as widgets
from IPython.display import display

# CZML Dynamic Update Demo

This notebook demonstrates how to dynamically update CZML entity properties using interactive Jupyter sliders.

Perfect for adjusting photo orientations, camera angles, or any CZML entity property in real-time!

## 🚀 Quick Start

1. **Run all cells** in order (Cell → Run All)
2. **Wait** for the CZML to load (~2 seconds)
3. **Open Browser Console** (F12) to see debug logs
4. **Use the sliders** to adjust orientation and position in real-time

## 💡 Important Notes

- The widget needs a moment to load CZML data before updates work
- Debug mode shows detailed logs in the browser console
- If sliders don't work immediately, wait a few seconds and try again

## Create a CZML Document with a Photo/Billboard Entity

We'll create a simple photo entity with position and orientation that we can manipulate with sliders.

In [2]:
# Sample CZML document with a simple cube/box
czml_data = [
    {
        "id": "document",
        "name": "Dynamic Photo Update Demo",
        "version": "1.0"
    },
    {
        "id": "photo1",
        "name": "Sample Cube",
        "position": {
            "cartographicDegrees": [-122.4175, 37.655, 100]  # lon, lat, alt
        },
        "orientation": {
            "unitQuaternion": [0, 0, 0, 1]  # Default orientation (no rotation)
        },
        "box": {
            "dimensions": {
                "cartesian": [40, 40, 40]  # width, depth, height in meters
            },
            "material": {
                "solidColor": {
                    "color": {
                        "rgba": [255, 0, 0, 255]  # Red color
                    }
                }
            },
            "fill": True,
            "outline": True,
            "outlineColor": {
                "rgba": [255, 255, 255, 255]  # White outline
            },
            "outlineWidth": 2
        }
    }
]

## Initialize the CesiumWidget with CZML Data

In [8]:
# Create widget and load CZML data
widget = CesiumWidget(height="200px", show_globe=True)
widget.load_czml(czml_data)

# Position camera to view the entity
widget.fly_to(37.655, -122.4175, altitude=500, pitch=-45, duration=2.0)

display(widget)

WARNING - cesiumjs_anywidget.widget - No Cesium Ion access token provided.
WARNING - cesiumjs_anywidget.widget - Your access token can be found at: https://ion.cesium.com/tokens
WARNING - cesiumjs_anywidget.widget - You can set it via:
WARNING - cesiumjs_anywidget.widget -   - CesiumWidget(ion_access_token='your_token')
WARNING - cesiumjs_anywidget.widget -   - export CESIUM_ION_TOKEN='your_token'  # in your shell
WARNING - cesiumjs_anywidget.widget - Note: Some features may not work without a token.


## Create Interactive Sliders for Orientation

Now let's create sliders to dynamically update the orientation (heading, pitch, roll) of our photo entity.

In [4]:
# Create sliders for heading, pitch, and roll
heading_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=360,
    step=1,
    description='Heading:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    # readout_format='.0f°'
)

pitch_slider = widgets.FloatSlider(
    value=0,
    min=-90,
    max=90,
    step=1,
    description='Pitch:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    # readout_format='.0f°'
)

roll_slider = widgets.FloatSlider(
    value=0,
    min=-180,
    max=180,
    step=1,
    description='Roll:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    # readout_format='.0f°'
)

# Function to update the entity orientation
def update_orientation(change=None):
    widget.update_czml_entity('photo1', {
        'orientation': {
            'heading': heading_slider.value,
            'pitch': pitch_slider.value,
            'roll': roll_slider.value
        }
    })

# Attach the update function to slider changes
heading_slider.observe(update_orientation, names='value')
pitch_slider.observe(update_orientation, names='value')
roll_slider.observe(update_orientation, names='value')

# Display sliders
display(widgets.VBox([
    widgets.HTML("<h3>Adjust Photo Orientation</h3>"),
    heading_slider,
    pitch_slider,
    roll_slider
]))

## Create Sliders for Position

You can also dynamically update the position of the entity.

In [5]:
# Create sliders for position
latitude_slider = widgets.FloatSlider(
    value=37.655,
    min=37.6,
    max=37.7,
    step=0.001,
    description='Latitude:',
    continuous_update=False,  # Update only when slider is released
    orientation='horizontal',
    readout=True,
    # readout_format='.3f°'
)

longitude_slider = widgets.FloatSlider(
    value=-122.4175,
    min=-122.5,
    max=-122.3,
    step=0.001,
    description='Longitude:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    # readout_format='.3f°'
)

altitude_slider = widgets.FloatSlider(
    value=100,
    min=0,
    max=500,
    step=10,
    description='Altitude:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    # readout_format='.0fm'
)

# Function to update the entity position
def update_position(change=None):
    widget.update_czml_entity('photo1', {
        'position': {
            'latitude': latitude_slider.value,
            'longitude': longitude_slider.value,
            'altitude': altitude_slider.value
        }
    })

# Attach the update function to slider changes
latitude_slider.observe(update_position, names='value')
longitude_slider.observe(update_position, names='value')
altitude_slider.observe(update_position, names='value')

# Display sliders
display(widgets.VBox([
    widgets.HTML("<h3>Adjust Photo Position</h3>"),
    latitude_slider,
    longitude_slider,
    altitude_slider
]))

## Combined Control Panel

Let's create a comprehensive control panel with all controls together.

In [6]:
# Create a reset button
reset_button = widgets.Button(
    description='Reset to Default',
    button_style='warning',
    tooltip='Reset all values to default',
    icon='refresh'
)

def reset_values(b):
    heading_slider.value = 0
    pitch_slider.value = 0
    roll_slider.value = 0
    latitude_slider.value = 37.655
    longitude_slider.value = -122.4175
    altitude_slider.value = 100

reset_button.on_click(reset_values)

# Create the full control panel
control_panel = widgets.VBox([
    widgets.HTML("<h2>📸 Photo Entity Control Panel</h2>"),
    widgets.HTML("<hr>"),
    widgets.HTML("<h3>🔄 Orientation</h3>"),
    heading_slider,
    pitch_slider,
    roll_slider,
    widgets.HTML("<hr>"),
    widgets.HTML("<h3>📍 Position</h3>"),
    latitude_slider,
    longitude_slider,
    altitude_slider,
    widgets.HTML("<hr>"),
    reset_button
])

display(control_panel)

## Advanced: Update Multiple Properties at Once

You can also update multiple properties simultaneously.

In [7]:
# Update both position and orientation at once
widget.update_czml_entity('photo1', {
    'orientation': {
        'heading': 45,
        'pitch': -30,
        'roll': 0
    },
    'position': {
        'latitude': 37.66,
        'longitude': -122.42,
        'altitude': 200
    }
})

print("Updated both position and orientation!")

Updated both position and orientation!


## Working with Real Photo Overlays

For actual photo projection use cases, you would typically:

1. Load a CZML document with your photo positions and initial orientations
2. Use sliders to fine-tune the orientation (heading, pitch, roll)
3. Optionally adjust position (latitude, longitude, altitude)
4. Save the final values back to your CZML document

Here's a helper function to get the current state:

In [8]:
def get_current_values():
    """Get current slider values as a dictionary."""
    return {
        'orientation': {
            'heading': heading_slider.value,
            'pitch': pitch_slider.value,
            'roll': roll_slider.value
        },
        'position': {
            'latitude': latitude_slider.value,
            'longitude': longitude_slider.value,
            'altitude': altitude_slider.value
        }
    }

# Get and display current values
current_values = get_current_values()
print("Current values:")
print(f"Heading: {current_values['orientation']['heading']}°")
print(f"Pitch: {current_values['orientation']['pitch']}°")
print(f"Roll: {current_values['orientation']['roll']}°")
print(f"Latitude: {current_values['position']['latitude']}°")
print(f"Longitude: {current_values['position']['longitude']}°")
print(f"Altitude: {current_values['position']['altitude']}m")

Current values:
Heading: 0.0°
Pitch: 0.0°
Roll: 0.0°
Latitude: 37.655°
Longitude: -122.4175°
Altitude: 100.0m


## Tips

- **Continuous Update**: Set `continuous_update=True` for real-time updates as you drag the slider
- **Discrete Update**: Set `continuous_update=False` to update only when you release the slider (better for performance)
- **Entity ID**: Make sure your entity ID in CZML matches the ID used in `update_czml_entity()`
- **Properties**: You can update any CZML entity property that Cesium supports
- **Debugging**: Enable debug mode with `widget.enable_debug()` to see update events in the browser console